# Understanding the output of LigandSwap

ligandswap is fully automatic. This means that you don’t need to do anything to get the calculation to run. However, ligandswap will print a lot of output to let you know what it is doing, and to give you a chance to check that everything is ok.

Because ligandswap calculations take a long time, we have provided some example output for you from this simulation. This is in `example_output.tar.bz2` and can be unpacked using the cell below.

In [ ]:
! tar -jxvf example_output.tar.bz2

The output printed when the simulation is running has been captured to a file called `runsim.log`. Take a look at it in the cell below.

In [ ]:
! cat runsim.log

The output from ligandswap should begin with something like;

```
Ligand 0 will be located by finding the first molecule containing residue FM1
Ligand 1 will be located by finding the first molecule containing residue CTI

Running a ligandswap calculation calculating the difference in free energy between
ligands 0 and 1 using files rec_fmc.top|fmc.30.crd and rec_cti.top|cti.30.crd.
Using parameters:
===============
crdfile0 == fmc.30.crd
crdfile1 == cti.30.crd
ligand0 == FM1
ligand1 == CTI
topfile0 == rec_fmc.top
topfile1 == rec_cti.top
===============
```

The above lines show that ligandswap has confirmed the parameters with which you have setup the simulation. Next, the output shows that ligandswap has loaded up the protein-ligand complex from rec_fmc.top and fmc.30.crd.

```
Loading the molecules from the files "fmc.30.crd" and "rec_fmc.top"...
Number of molecules == 44095
System space == PeriodicBox( ( 120.212, 116.603, 105.492 ) )
...
Only moving molecules/residues that are within a distance 15.0 A of the point ( 0, 0, 0 ).
```

The whole system is loaded, but as the last line shows, to keep the computational cost down, only molecules that are within 15 angstroms of the ligand (FM1) are moved.

Next, the CTI ligand is loaded from its topology and coordinate files…

```
Loading from Amber files rec_cti.top / cti.30.crd...
Loading the molecules from the files "cti.30.crd" and "rec_cti.top"...
Number of molecules == 44310
System space == PeriodicBox( ( 120.283, 116.671, 105.917 ) )
...
Only moving molecules/residues that are within a distance 15.0 A of the point ( 0, 0, 0 ).
```

Once this has been loaded, the next step is to load up an empty, equilibrated box of water. This is the box in which the free ligand will be placed. Sire comes with its own pre-equilibrated box of water, which is loaded next…

```
Loading the molecules from the files "/home/chris/sire.app/pkgs/sire-2017.3.0/share/Sire/Tools/WSRC/waterbox.crd" and "/home/chris/sire.app/pkgs/sire-2017.3.0/share/Sire/Tools/WSRC/waterbox.top"...
Number of molecules == 35394
System space == PeriodicBox( ( 102.562, 102.637, 102.219 ) )
```

Once all of the molecules have been loaded, the next step is to combine them all together into the ligandswap system. This involves building two simulation boxes;

* a “protein box”, containing the protein in water,
* and a “water box” containing just water

Both ligands FM1 and CTI are placed in both boxes, and a λ-coordinate created that will swap these two ligands between the protein box and water box.

```
Merging the two ligand complexes with the water system to create the ligandswap system...
```

For this swap to work, the second ligand (CTI) has to be translated and rotated so that it is aligned on top of the first ligand (FM1). This is to make sure that, as CTI is swapped in, it has the best chance of making favourable interactions with the protein, and that it causes the minimum amount of disruption to the protein binding site.

There are many ways to align molecules. ligandswap uses a “maximum common substructure” alignment. The program looks at the connectivity (bonding) of the atoms to find the largest common substructure (framework / scaffold) of the two ligands. The second ligand is then aligned onto the first such that the root mean square deviation (RMSD) between equivalent atoms on this common scaffold is minimised. The below lines are printed when ligandswap is finding the maximum common substructure…

```
First, mapping the atoms from the first ligand to the atoms of the second...
Mapping:
C1 <=> C1
C12 <=> C12
C17 <=> C17
C18 <=> C18
C19 <=> C19
C2 <=> C2
C21 <=> C22
C28 <=> C20
C3 <=> C3
C6 <=> C6
C9 <=> C9
N23 <=> N25
N25 <=> N27
O24 <=> O26


PLEASE CHECK: Writing alignment of ligands to the file aligned_ligands.pdb.
PLEASE CHECK: View this file in a PDB viewer to check that the ligands are aligned.
```

After the ligands have been aligned, their coordinates are printed out into the file aligned_ligands.pdb. You should always look at this file, e.g. in VMD, to double-check that ligandswap has properly aligned the two ligands. 

We can do this in the notebook by executing the cell below

In [ ]:
import BioSimSpace as BSS
BSS.viewMolecules("aligned_ligands.pdb")

The molecules are overlaid, hence the weird bonding at the back. Despite this, you should see that the two molecules are on top of each other, with the fluromethyl group of FM1 on top of the the hydrogen of CTI. If you want a clearer view, you can download the file and load it into VMD.

![](images/aligned.jpg)

For example, the figure above shows a snapshot from VMD of aligned_ligands.pdb. In this, you can see that swapping FM1 with CTI will just involve losing the flouromethyl group and replacing it with a hydrogen. This swap does not involve a large change in the shape or volume occupied by the ligands in the protein binding site, so there is a good chance that the ligandswap free energy calculation will converge well and give a more robust prediction of relative binding free energy.

If the alignment isn't correct, then you can add an optional match string to the ligandswap command to tell it which atoms are equivalent in the two ligands. The format of the match string is a comma-separated set of pairs of atom names, e.g. `A1:B1,A2:B2,A3:B3`. This says that the atom called `A1` is the first ligand is equivalent to the atom called `B1` in the second ligand, that `A2` is equivalent to `B2`, and `A3` is equivalent to `B3`.

For example, if we wanted to force ligandswap to match the atoms called `C1`, `C2` and `C3` in FMC to `C1`, `C2` and `C5` in CTI, then the match string would be `-m C1:C1,C2:C2,C3:C5`, and the full ligandswap command would be

```
$SIRE/bin/ligandswap -t0 rec_fmc.top -c0 fmc.30.crd -l0 FM1 -t1 rec_cti.top -c1 cti.30.crd -l1 CTI -m C1:C1,C2:C2,C3:C5
```